In [40]:
import sys
sys.path.append("..")

from libs.InteractivePitchUI import *
from libs.data_manipulation import *
from libs.dim_reduction import *
from libs.feature_generation import *
from libs.clustering import *
from libs.convex_hull import *
from libs.alpha_shape import *
from libs.weight_generator import *
from libs.similar_movement import *

In [44]:
import os
import pandas as pd
import os
import pandas as pd
import os
import pandas as pd

def compile_situations_from_folder(folder_path):
    """
    Compile all CSV files in the given folder into a list of DataFrames, maintaining their order in the folder.

    Parameters:
    - folder_path (str): Path to the folder containing CSV files.

    Returns:
    - List of DataFrames, in the same order as the files are listed in the folder.
    """
    # List to store DataFrames
    dataframes = []

    # Get a list of all files in the directory and sort them
    csv_files = sorted([f for f in os.listdir(folder_path) if f.endswith('.csv')])

    # Iterate through the sorted CSV files
    for filename in csv_files:
        # Create the full path to the file
        file_path = os.path.join(folder_path, filename)

        # Read the CSV file into a DataFrame
        df = pd.read_csv(file_path)

        # Apply any necessary data processing here (e.g., converting the "Unnamed: 0" column)
        if "Unnamed: 0" in df.columns:
            df["Unnamed: 0"] = df["Unnamed: 0"].apply(lambda x: int(x))

        # Append the DataFrame to the list
        dataframes.append(df)

    return dataframes



# Example usage
folder = "control"  # Replace with the actual folder path
situations_list = compile_situations_from_folder(folder)


In [45]:
sit_1_results = compile_situations_from_folder("rankings/breakthrough-even_Denmark_England_0:46:08")
sit_2_results = compile_situations_from_folder("rankings/breakthrough-space_Denmark_England_0:30:51")
sit_3_results = compile_situations_from_folder("rankings/breakthrough-wb_Denmark_Serbia_0:17:18")
sit_4_results = compile_situations_from_folder("rankings/cross-from-cb_Denmark_Serbia_0:41:40")

In [49]:

print(situations_list[0])

   Unnamed: 0        match_name         rating  score
0      418224    Denmark_Serbia       Ikke god      0
1      449280   Germany_Denmark             OK      1
2       31152  Slovenia_Denmark  Slet ikke god      0
3       63744  Slovenia_Denmark            God      3


In [63]:


#resulting_dfs_1 = [pd.merge(situations_list[0], weighting_result, on='Unnamed: 0', how='left') for weighting_result in sit_1_results]
resulting_dfs_2 = [pd.merge(situations_list[1], weighting_result, on='Unnamed: 0', how='left') for weighting_result in sit_2_results]
resulting_dfs_3 = [pd.merge(situations_list[2], weighting_result, on='Unnamed: 0', how='left') for weighting_result in sit_3_results]
resulting_dfs_4 = [pd.merge(situations_list[3], weighting_result, on='Unnamed: 0', how='left') for weighting_result in sit_4_results]


resulting_dfs_2[0].score
resulting_dfs_2


[   Unnamed: 0     match_name_x    rating  score     match_name_y  \
 0      252384  Denmark_England  Ikke god      0  Denmark_England   
 1      272784  Denmark_England        OK      1  Denmark_England   
 2      304368   Denmark_Serbia  Ikke god      0   Denmark_Serbia   
 3      385392   Denmark_Serbia        OK      1   Denmark_Serbia   
 4      390288   Denmark_Serbia  Ikke god      0   Denmark_Serbia   
 
          Time [s] half  
 0  1:11:35.840000   2H  
 1  1:25:11.840000   2H  
 2  0:13:08.160000   1H  
 3  1:06:10.840000   2H  
 4  1:09:26.680000   2H  ,
    Unnamed: 0     match_name_x    rating  score     match_name_y  \
 0      252384  Denmark_England  Ikke god      0  Denmark_England   
 1      272784  Denmark_England        OK      1  Denmark_England   
 2      304368   Denmark_Serbia  Ikke god      0   Denmark_Serbia   
 3      385392   Denmark_Serbia        OK      1   Denmark_Serbia   
 4      390288   Denmark_Serbia  Ikke god      0   Denmark_Serbia   
 
          T

In [64]:
# Function to calculate nDCG
def calculate_ndcg(df, score_column, k=None):
    if k is None:
        k = len(df)

    # Extract relevance scores (i.e., the score column values)
    scores = df[score_column].values[:k]

    # Compute DCG (Discounted Cumulative Gain)
    dcg = np.sum((2**scores - 1) / np.log2(np.arange(2, k + 2)))

    # Compute IDCG (Ideal DCG)
    sorted_scores = np.sort(scores)[::-1]  # Sort in descending order
    idcg = np.sum((2**sorted_scores - 1) / np.log2(np.arange(2, k + 2)))

    # Return nDCG (normalized DCG)
    return dcg / idcg if idcg > 0 else 0.0

# Function to calculate nDCG for each situation (Sit_2, Sit_3, Sit_4)
def calculate_ndcg_for_situations(situations_dfs, methods):
    ndcg_scores = {}
    
    # Iterate over each situation DataFrame list (Sit_2, Sit_3, Sit_4)
    for i, situation_df_list in enumerate(situations_dfs):
        situation_key = f"Sit_{i + 2}"  # Sit_2, Sit_3, Sit_4
        ndcg_scores[situation_key] = {}
        
        # Iterate over each method (Inverse Weighting, Linear Weighting, etc.)
        for j, method in enumerate(methods):
            # Calculate nDCG for the current method applied to the current situation
            df = situation_df_list[j]
            ndcg_scores[situation_key][method] = calculate_ndcg(df, score_column='score')  # Assuming the scores are in 'score' column
    
    return ndcg_scores

# Define methods (assuming columns are named exactly as the weighting methods)
methods = ["Inverse Weighting", "Linear Weighting", "Control", "Exponential Weighting"]

# List of DataFrames for each situation (Sit_2, Sit_3, Sit_4)
situations_dfs = [resulting_dfs_2, resulting_dfs_3, resulting_dfs_4]

# Calculate nDCG scores for all situations
ndcg_scores = calculate_ndcg_for_situations(situations_dfs, methods)

# Print the nDCG scores to verify the calculation
print(ndcg_scores)


{'Sit_2': {'Inverse Weighting': 0.6509209298071326, 'Linear Weighting': 0.6509209298071326, 'Control': 0.6509209298071326, 'Exponential Weighting': 0.6509209298071326}, 'Sit_3': {'Inverse Weighting': 1.0, 'Linear Weighting': 1.0, 'Control': 1.0, 'Exponential Weighting': 1.0}, 'Sit_4': {'Inverse Weighting': 0.8503449055347546, 'Linear Weighting': 0.8503449055347546, 'Control': 0.8503449055347546, 'Exponential Weighting': 0.8503449055347546}}


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Extract nDCG scores for plotting
ndcg_values = {
    "Sit_2": [],
    "Sit_3": [],
    "Sit_4": [],
}

# Extract values for plotting
for sit_key, sit_scores in ndcg_scores.items():
    for method in methods:
        ndcg_values[sit_key].append(sit_scores[method])

# Calculate the average nDCG for each method across the situations
average_ndcg = np.mean([ndcg_values['Sit_2'], ndcg_values['Sit_3'], ndcg_values['Sit_4']], axis=0)

# Define positions for the bars, grouped by method
n_methods = len(methods)
group_width = 0.25  # width of each group (bar width)
x_pos = np.arange(n_methods)  # Positions for the bars on the x-axis

# Plot the nDCG scores for all situations grouped by metric
plt.figure(figsize=(12, 6))
plt.bar(x_pos - group_width, ndcg_values['Sit_2'], width=group_width, label="Sit_2", color=['blue', 'green', 'orange', 'red'])
plt.bar(x_pos, ndcg_values['Sit_3'], width=group_width, label="Sit_3", color=['purple', 'cyan', 'brown', 'pink'])
plt.bar(x_pos + group_width, ndcg_values['Sit_4'], width=group_width, label="Sit_4", color=['yellow', 'gray', 'indigo', 'violet'])

# Plot the average nDCG as a marker or line
plt.plot(x_pos, average_ndcg, color='black', marker='o', linestyle='--', label="Average nDCG", linewidth=2)

# Add labels, title, and customize axes
plt.title("nDCG Comparison for Sit_2, Sit_3, and Sit_4 by Metric", fontsize=14)
plt.ylabel("nDCG", fontsize=12)
plt.xlabel("Methods", fontsize=12)
plt.ylim(0, 1)  # nDCG ranges from 0 to 1

# Customizing tick positions and labels
plt.xticks(x_pos, methods, rotation=45, ha='right')
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.legend(title="Situation", loc='upper left')
plt.tight_layout()  # Ensure the labels fit well
plt.show()
